# Stratification validation using ICES temperature

In [ ]:
chunk_start

Vertical profiles of temperature were extracted from the ICES oceanographic database. The dataset was pruned to remove profiles with poor quality vertical coverage. First, we removed any profiles without temperature for the top 5 metres. We then removed profiles with less than 10 temperature measurements. Finally, we removed profiles which did not have a temperature record deeper than 80% of the bathymetry. 

In [ ]:
import seawater as sw
ff = glob.glob("../../matched/point/**/all/temperature/*temperature*.csv")[0]
df = pd.read_csv(ff)
df["pressure"] = sw.eos80.pres(df.depth, df.lat)


In [ ]:
## work out whether model/observations are stratified
# stratification is defined when the difference between the surface and bottom temperature is greater than 0.5 degrees

df_strat = (
    profiles
    .merge(df)
    .merge(profile_mld.loc[:,["lon", "lat", "year", "month", "day"]])
    .drop(columns = "pressure")
    .melt(id_vars = ["lon", "lat", "day", "month", "year", "depth"])
    .rename(columns = {"value": "temperature", "variable": "source"})
    .groupby(["lon", "lat", "day", "month", "year", "source"])
    .agg({"temperature": ["min", "max"]})
    .reset_index()
    .assign(stratified = lambda x: x.temperature["max"] - x.temperature["min"] > 0.5)
    .drop(columns = "temperature")
)
df_strat = df_strat.reset_index(drop = True)
df_strat.columns = [x[0] for x in list(df_strat.columns)]

## Accuracy of stratification

We first identify whether the model and observations are stratified or not. This uses a simple metric of the difference between the surface and bottom temperature. If this is greater than 0.5 degrees, then the profile is considered stratified.



In [ ]:
%%capture --no-display
%%R -i df_strat -w 1600 -h 800

if(nrow(df_strat) > 0){
# increase plot size

library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df_strat$lon), max(df_strat$lon))
ylim = c(min(df_strat$lat), max(df_strat$lat))

# cap value at 100

# convert month to Jan, Feb, etc.
# df_strat$month <- factor(df_strat$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))
df_strat <- df_strat %>%
    arrange(month)
df_strat$month <- factor(df_strat$month, levels = df_strat$month, labels = month.abb[df_strat$month])


gg <- df_strat %>%
# # # first six months of the year
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun")) %>%
    ggplot()+
    geom_point(aes(lon, lat, colour = stratified), size = 0.5)+
    facet_grid(source~month)+
    theme_gray(base_size = 28)+
#     # add colour scale. Minimum zero, label 100, ">100"
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # red-blue colour scale
    scale_color_manual(values = c("blue", "red"))+
    # no legend
    theme(legend.position = "none")


y_labels <-  as.numeric(na.omit(layer_scales(gg)$y$break_positions()))
x_labels <- as.numeric(na.omit(layer_scales(gg)$x$break_positions()))
x_breaks <- x_labels
y_breaks <- y_labels

# y labels are north-south coordinates. Make them more appropriate
# i.e. 10 should be 10 °N, -10 should be 10 °S

y_labels <- ifelse(y_labels >= 0, paste0(y_labels, "°N"), paste0(abs(y_labels), "°S"))
x_labels <- ifelse(x_labels >= 0, paste0(x_labels, "°E"), paste0(abs(x_labels), "°W"))

gg <- gg + scale_y_continuous(breaks = y_breaks, labels = y_labels)+
    scale_x_continuous(breaks = x_breaks, labels = x_labels)+
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    labs(x = "", y = "")

    # move legen

gg

}

In [ ]:
%%capture --no-display
%%R -i df_strat -w 1600 -h 800
# increase plot size
if(nrow(df_strat) > 0){
library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df_strat$lon), max(df_strat$lon))
ylim = c(min(df_strat$lat), max(df_strat$lat))

# cap value at 100

# convert month to Jan, Feb, etc.
# df_strat$month <- factor(df_strat$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))
df_strat <- df_strat %>%
    arrange(month)
df_strat$month <- factor(df_strat$month, levels = df_strat$month, labels = month.abb[df_strat$month])


gg <- df_strat %>%
# # # first six months of the year
#final six months of the year
    filter(month %in% c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec")) %>%
    ggplot()+
    geom_point(aes(lon, lat, colour = stratified), size = 0.5)+
    facet_grid(source~month)+
    theme_gray(base_size = 28)+
#     # add colour scale. Minimum zero, label 100, ">100"
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # red-blue colour scale
    scale_color_manual(values = c("blue", "red"))+
    # no legend
    theme(legend.position = "none")

y_labels <-  as.numeric(na.omit(layer_scales(gg)$y$break_positions()))
x_labels <- as.numeric(na.omit(layer_scales(gg)$x$break_positions()))
x_breaks <- x_labels
y_breaks <- y_labels

# y labels are north-south coordinates. Make them more appropriate
# i.e. 10 should be 10 °N, -10 should be 10 °S

y_labels <- ifelse(y_labels >= 0, paste0(y_labels, "°N"), paste0(abs(y_labels), "°S"))
x_labels <- ifelse(x_labels >= 0, paste0(x_labels, "°E"), paste0(abs(x_labels), "°W"))

gg <- gg + scale_y_continuous(breaks = y_breaks, labels = y_labels)+
    scale_x_continuous(breaks = x_breaks, labels = x_labels)+
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")

gg <- gg +
    labs(x = "", y = "")

    # move legen

gg

}

In [ ]:
md(f"**Figure {i_figure}:** Stratification of the water column. Red indicates stratified, blue indicates unstratified. Stratification is defined based the difference between the sea surface and bottom temperature being greater than 0.5 degrees.")
i_figure += 1

In [ ]:
gg = (
    ggplot(df_strat.groupby(["source", "month"]).agg({"stratified": "mean"}).reset_index())+
    geom_line(aes("month", "stratified", colour = "source"))+
    labs(x = "Month", y = "Proportion of profiles stratified")
)
gg = gg.draw()
gg


In [ ]:
md(f"**Figure {i_figure}**: Proportion of profiles stratified by month in the model and ICES profiles.")
i_figure += 1

## Data Sources

In [ ]:
md("ICES Data Portal, Dataset on Ocean HydroChemistry, Extracted March 3, 2023. ICES, Copenhagen")

In [ ]:
chunk_end